In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 199 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=abfd07c7ce58cc7e3fbd9f6841fce17cdfea7aeebae16e76255665f5a6753f59
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [54]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [55]:
spark = SparkSession.builder.appName("Customers").getOrCreate()

In [56]:
path = "Final Dataset (1).csv"

df = spark.read.option("header",'True').option('delimiter', ',').csv(path)
df.printSchema()

root
 |-- group: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gendera: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- hypertensive: string (nullable = true)
 |-- atrialfibrillation: string (nullable = true)
 |-- CHD with no MI: string (nullable = true)
 |-- diabetes: string (nullable = true)
 |-- deficiencyanemias: string (nullable = true)
 |-- depression: string (nullable = true)
 |-- Hyperlipemia: string (nullable = true)
 |-- Renal failure: string (nullable = true)
 |-- COPD: string (nullable = true)
 |-- heart rate: string (nullable = true)
 |-- Systolic blood pressure: string (nullable = true)
 |-- Diastolic blood pressure: string (nullable = true)
 |-- Respiratory rate: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- SP O2: string (nullable = true)
 |-- Urine output: string (nullable = true)
 |-- hematocrit: string (nullable = true)
 |-- RBC: str

In [57]:
df.show(5)

+-----+------+-------+---+-------+-----------+------------+------------------+--------------+--------+-----------------+----------+------------+-------------+----+-----------+-----------------------+------------------------+----------------+-----------+-----------+------------+-----------+-----------+-----------+-----------+------+-----------+-----------+---------+-----------+---------+----------+-----------+-----------+---------+---------------+-----------+-------------+-----------+---------------+------------+-------------+-----------+-----------+-------------+-----+-----------+-----------+----+---+
|group|    ID|outcome|age|gendera|        BMI|hypertensive|atrialfibrillation|CHD with no MI|diabetes|deficiencyanemias|depression|Hyperlipemia|Renal failure|COPD| heart rate|Systolic blood pressure|Diastolic blood pressure|Respiratory rate|temperature|      SP O2|Urine output| hematocrit|        RBC|        MCH|       MCHC|   MCV|        RDW|  Leucocyte|Platelets|Neutrophils|Basophils|Ly

In [95]:
##subsetting data frame
df1 = df.select(df['Renal failure'],df['Systolic blood pressure'],df['Blood potassium'],df['heart rate'],df['hematocrit'],df['Urine output'],df['Urea nitrogen'],df['Creatinine'],df['hypertensive'],df['diabetes'],df['deficiencyanemias'],df['Lactic acid'],df['EF'])
                                            

In [97]:
from pyspark.sql.types import IntegerType, FloatType, StringType
df = df.withColumn("Renal failure", df["Renal failure"].cast(IntegerType()))\
.withColumn("Systolic blood pressure", df["Systolic blood pressure"].cast(FloatType()))\
.withColumn("Blood potassium", df["Blood potassium"].cast(FloatType()))\
.withColumn("heart rate", df["heart rate"].cast(FloatType()))\
.withColumn("hematocrit", df["hematocrit"].cast(FloatType()))\
.withColumn("Urine output", df["Urine output"].cast(IntegerType()))\
.withColumn("Urea nitrogen", df["Urea nitrogen"].cast(FloatType()))\
.withColumn("Creatinine", df["Creatinine"].cast(FloatType()))\
.withColumn("hypertensive", df["hypertensive"].cast(IntegerType()))\
.withColumn("diabetes", df["diabetes"].cast(IntegerType()))\
.withColumn("deficiencyanemias", df["deficiencyanemias"].cast(IntegerType()))\
.withColumn("Lactic acid", df["Lactic acid"].cast(FloatType()))\
.withColumn("EF", df["EF"].cast(IntegerType()))

In [98]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Systolic blood pressure','Blood potassium','heart rate','hematocrit','Urine output','Urea nitrogen','Creatinine','hypertensive','diabetes','deficiencyanemias','Lactic acid','EF'], 
    outputCols=["{}".format(c) for c in ['Systolic blood pressure','Blood potassium','heart rate','hematocrit','Urine output','Urea nitrogen','Creatinine','hypertensive','diabetes','deficiencyanemias','Lactic acid','EF']]
)
df= imputer.fit(df).transform(df)


In [99]:
featureassembler = VectorAssembler(inputCols = ['Systolic blood pressure','Blood potassium','heart rate','hematocrit','Urine output','Urea nitrogen','Creatinine','hypertensive','diabetes','deficiencyanemias','Lactic acid','EF'], outputCol = "features")

In [100]:
output = featureassembler.transform(df)
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[155.866668701171...|
|[140.0,4.44999980...|
|[135.333328247070...|
|[126.400001525878...|
|[156.559997558593...|
|[118.099998474121...|
|[106.565216064453...|
|[141.130432128906...|
|[98.4347839355468...|
|[122.0,3.86999988...|
|[149.035720825195...|
|[103.260871887207...|
|[126.903228759765...|
|[112.142860412597...|
|[107.360000610351...|
|[159.695648193359...|
|[157.289474487304...|
|[113.279998779296...|
|[162.240005493164...|
|[112.416664123535...|
+--------------------+
only showing top 20 rows



In [101]:
finalised_data = output.select("features", "Renal failure")

In [102]:
finalised_data = output.select('features', 'Renal failure')
finalised_data.show()

+--------------------+-------------+
|            features|Renal failure|
+--------------------+-------------+
|[155.866668701171...|            1|
|[140.0,4.44999980...|            0|
|[135.333328247070...|            1|
|[126.400001525878...|            0|
|[156.559997558593...|            1|
|[118.099998474121...|            1|
|[106.565216064453...|            1|
|[141.130432128906...|            0|
|[98.4347839355468...|            1|
|[122.0,3.86999988...|            0|
|[149.035720825195...|            0|
|[103.260871887207...|            0|
|[126.903228759765...|            0|
|[112.142860412597...|            1|
|[107.360000610351...|            0|
|[159.695648193359...|            1|
|[157.289474487304...|            1|
|[113.279998779296...|            1|
|[162.240005493164...|            0|
|[112.416664123535...|            1|
+--------------------+-------------+
only showing top 20 rows



In [103]:
train, test = finalised_data.randomSplit([0.7, 0.3])

In [104]:
lr = LogisticRegression(labelCol="Renal failure")
lrn = lr.fit(train)

In [105]:
lrn_summary = lrn.summary
lrn_summary.predictions.show()

+--------------------+-------------+--------------------+--------------------+----------+
|            features|Renal failure|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|[82.4545440673828...|          0.0|[1.77398290334447...|[0.85495228502942...|       0.0|
|[85.0588226318359...|          0.0|[1.97328291057122...|[0.87796329319876...|       0.0|
|[85.28125,4.01999...|          0.0|[0.94022621683316...|[0.71914534989764...|       0.0|
|[85.3571395874023...|          0.0|[4.29316355506079...|[0.98652248737285...|       0.0|
|[85.5909118652343...|          1.0|[0.57698971671715...|[0.64037444644685...|       0.0|
|[85.8857116699218...|          1.0|[0.79125110474764...|[0.68809990388378...|       0.0|
|[85.9523773193359...|          0.0|[1.28463962959110...|[0.78323850900653...|       0.0|
|[86.2608718872070...|          1.0|[-3.0538211787038...|[0.04505278868512...|       1.0|
|[86.63636

In [106]:
lrn_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|      Renal failure|         prediction|
+-------+-------------------+-------------------+
|  count|                793|                793|
|   mean| 0.3379571248423707|0.25220680958385877|
| stddev|0.47331238022104516|0.43455340781830276|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [107]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [108]:
lrn_summary.predictions.show()
eval = BinaryClassificationEvaluator(rawPredictionCol = "prediction", labelCol = "Renal failure")
auc = eval.evaluate(lrn_summary.predictions)
print(auc)

+--------------------+-------------+--------------------+--------------------+----------+
|            features|Renal failure|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+--------------------+----------+
|[82.4545440673828...|          0.0|[1.77398290334447...|[0.85495228502942...|       0.0|
|[85.0588226318359...|          0.0|[1.97328291057122...|[0.87796329319876...|       0.0|
|[85.28125,4.01999...|          0.0|[0.94022621683316...|[0.71914534989764...|       0.0|
|[85.3571395874023...|          0.0|[4.29316355506079...|[0.98652248737285...|       0.0|
|[85.5909118652343...|          1.0|[0.57698971671715...|[0.64037444644685...|       0.0|
|[85.8857116699218...|          1.0|[0.79125110474764...|[0.68809990388378...|       0.0|
|[85.9523773193359...|          0.0|[1.28463962959110...|[0.78323850900653...|       0.0|
|[86.2608718872070...|          1.0|[-3.0538211787038...|[0.04505278868512...|       1.0|
|[86.63636